## Imports

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import datetime
import os
import folium
from folium import plugins
import ast

from IPython.display import display
from queries import ride_segments_query

In [2]:
import os
os.environ["PATH"] += os.pathsep + "."

In [3]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import pandas as pd
import folium
from folium import plugins
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, rgb_to_hsv, hsv_to_rgb
import scipy.ndimage.filters
import time
import datetime
import os.path
import io

import os
os.environ["PATH"] += os.pathsep + "."

%matplotlib inline

## Tricks and tips

In [4]:
# geocoder

## Functions

In [4]:
def plot_circles(df, zoom_start=5, radius_scale=1000):
    
    # initialize map
    folium_map = folium.Map(location=[50.109900, 8.648916],
                        zoom_start=zoom_start,
                        tiles="CartoDB dark_matter")
    
    # draw circle for all rows (since we grab every segment this corresponds to segments)
    for index, row in df.iterrows():

        radius = row['capacity'] / radius_scale
        color = 'lime'

        marker = folium.CircleMarker(location=(
            row["from_lat"],
            row["from_long"]),
            radius=radius,
            color=color,
    #         popup=row['cap_per_bus'], does not plot
            opacity=0.1,
            fill_opacity=0.2,
            fill=True
            )

        marker.add_to(folium_map)
    
    return folium_map

## Get data

In [5]:
CCOPTI_PROD = os.environ['POSTGRES_CONN_STR_PROD']
pg_prod = sa.create_engine(CCOPTI_PROD)

In [6]:
rides = pd.read_sql(ride_segments_query, pg_prod)

In [7]:
rides.departure_time = [datetime.datetime.strptime(x, '%H:%M:%S') for x in rides['departure_time']]

In [8]:
# get dictionaries out of strings with ast and save coordinates
def get_long_lat(rides):
    rides['from_lat'] = rides['from_stop_geoc'].apply(lambda x: ast.literal_eval(f"""{x}""")['latitude'])
    rides['from_long'] = rides['from_stop_geoc'].apply(lambda x: ast.literal_eval(f"""{x}""")['longitude'])
    rides['to_lat'] = rides['to_stop_geoc'].apply(lambda x: ast.literal_eval(f"""{x}""")['latitude'])
    rides['to_long'] = rides['to_stop_geoc'].apply(lambda x: ast.literal_eval(f"""{x}""")['longitude'])
    
    return rides

rides = get_long_lat(rides)
rides

,ride_id,line_id,from_id,to_id,from_stop_geoc,to_stop_geoc,capacity,num_buses,departure_date,departure_time,from_lat,from_long,to_lat,to_long
0,79143561,5328,11548,24,"{""latitude"":50.881001,""longitude"":7.117833}","{""latitude"":51.032368284265,""longitude"":6.9910...",78.0,1,2020-01-01,1900-01-01 19:25:00,50.881001,7.117833,51.032368,6.991052
1,79143561,5328,52,49,"{""latitude"":51.450253,""longitude"":7.01461}","{""latitude"":51.51954,""longitude"":7.459387}",78.0,1,2020-01-01,1900-01-01 21:25:00,51.450253,7.014610,51.519540,7.459387
2,79143561,5328,30,251,"{""latitude"":51.345057312421,""longitude"":12.384...","{""latitude"":51.039905,""longitude"":13.730732}",78.0,1,2020-01-02,1900-01-01 03:35:00,51.345057,12.384724,51.039905,13.730732
3,79143561,5328,251,1494,"{""latitude"":51.039905,""longitude"":13.730732}","{""latitude"":50.089617,""longitude"":14.440946}",78.0,1,2020-01-02,1900-01-01 05:20:00,51.039905,13.730732,50.089617,14.440946
4,79143561,5328,49,71,"{""latitude"":51.51954,""longitude"":7.459387}","{""latitude"":51.477534,""longitude"":11.984321}",78.0,1,2020-01-01,1900-01-01 22:10:00,51.519540,7.459387,51.477534,11.984321
5,79143561,5328,71,30,"{""latitude"":51.477534,""longitude"":11.984321}","{""latitude"":51.345057312421,""longitude"":12.384...",78.0,1,2020-01-02,1900-01-01 02:40:00,51.477534,11.984321,51.345057,12.384724
6,79143561,5328,25,52,"{""latitude"":51.2228649,""longitude"":6.7954195}","{""latitude"":51.450253,""longitude"":7.01461}",78.0,1,2020-01-01,1900-01-01 20:40:00,51.222865,6.795420,51.450253,7.014610
7,79143561,5328,24,25,"{""latitude"":51.032368284265,""longitude"":6.9910...","{""latitude"":51.2228649,""longitude"":6.7954195}",78.0,1,2020-01-01,1900-01-01 19:55:00,51.032368,6.991052,51.222865,6.795420
8,80107371,1564,1554,89,"{""latitude"":53.211648,""longitude"":6.566523}","{""latitude"":53.082179,""longitude"":8.810576}",75.0,1,2020-01-01,1900-01-01 04:00:00,53.211648,6.566523,53.082179,8.810576
9,80107371,1564,36,1,"{""latitude"":53.551767,""longitude"":10.011657}","{""latitude"":52.507171,""longitude"":13.279399}",75.0,1,2020-01-01,1900-01-01 08:15:00,53.551767,10.011657,52.507171,13.279399


In [9]:
# group data to get capacity by departure date
grouped_cap = rides.groupby(['from_id', 'from_lat', 'from_long'], as_index=False).sum()
grouped_cap['cap_per_bus'] = grouped_cap.capacity / grouped_cap.num_buses
grouped_cap.sort_values('cap_per_bus', ascending=False)

,from_id,from_lat,from_long,ride_id,line_id,to_id,capacity,num_buses,to_lat,to_long,cap_per_bus
1323,22348,49.553281,8.665296,496298595,142655,103180,3300.0,5,247.956738,43.286182,660.000000
2086,37111,50.976984,10.332135,496298595,142655,136536,3300.0,5,253.541907,50.476270,660.000000
1281,20708,50.099857,8.685935,595558516,171186,133498,3960.0,6,301.699176,56.293832,660.000000
1278,20648,49.872516,8.629379,595558516,171186,78,3960.0,6,299.225513,51.771644,660.000000
1277,20628,49.403902,8.675808,595558516,171186,84,3960.0,6,296.436315,52.040504,660.000000
1276,20608,48.783206,9.182289,297781623,85593,61884,1980.0,3,148.211706,26.027424,660.000000
1661,30078,50.972454,11.038239,595558516,171186,4464,3960.0,6,305.842632,66.239822,660.000000
1662,30098,51.478526,11.987716,595558516,171186,426,3960.0,6,308.865204,71.905926,660.000000
1282,20738,50.554587,9.683728,496298595,142655,136346,3300.0,5,252.253539,46.722075,660.000000
2087,37161,50.939073,10.712543,297780263,85593,10614,1980.0,3,152.820549,32.137854,660.000000


## Cap per stop maps

### Base map: bubbles per stop

In [11]:
plot_circles(df=grouped_cap)
#folium_map.save('map.html')

### Heatmap

In [12]:
# initialize map
heat_map = folium.Map(location=[50.109900, 8.648916],
                        zoom_start=5,
                        tiles="CartoDB dark_matter")

# convert to (n, 2) nd-array format for heatmap
grouped_cap_arr = grouped_cap[['from_lat', 'from_long']].as_matrix()

# plot heatmap
heatmap.add_children(plugins.HeatMap(grouped_cap_arr, radius=10))
heatmap

/Users/davidbiermann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


NameError: name 'heatmap' is not defined

In [13]:
# initialize map
heat_map = folium.Map(location=[50.109900, 8.648916],
                        zoom_start=5,
                        tiles="CartoDB dark_matter")

lat_long_weight = []

for index, row in grouped_cap.iterrows():
    lat_long_weight.append(
        [row['from_lat'],
         row['from_long'],
         row['capacity']])
    
plugins.HeatMap(data=lat_long_weight,radius=10).add_to(heat_map)

heat_map

### heatmap with time

In [14]:
# example
# take note of data format needed for heat map with time (using plugins.HeatMapWithTime?)
heatmap_time_data = (np.random.random((100,20,2)) + np.array([[39, -98]])).tolist()

# dates 
heatmap_time_dates = heatmap_time_data.departure_time

# map
map_heatmap_time = folium.Map([39, -98], tiles='CartoDB Dark_Matter', zoom_start=6)

# heatmap plugin
heatmap_time_plugin = plugins.HeatMapWithTime(heatmap_time_data, index=heatmap_time_dates)

# add heatmap plugin to map
heatmap_time_plugin.add_to(map_heatmap_time)

# display map
map_heatmap_time.save('timeheat.html')

AttributeError: 'list' object has no attribute 'departure_time'

In [15]:
# group data by coords AND time
time_grouped_cap = rides.groupby(['from_id', 'from_lat', 'from_long', 'departure_time'], as_index=False).sum()

# initialize map
time_heat_map = folium.Map(location=[50.109900, 8.648916],
                        zoom_start=5,
                        tiles="CartoDB dark_matter")

lat_long_weight = []

for index, row in time_grouped_cap.iterrows():
    lat_long_weight.append(
        [row['from_lat'],
         row['from_long'],
         row['capacity']])

index = [datetime.strptime(x, '%H:%M:%S') for x in time_grouped_cap['departure_time']]

plugins.HeatMapWithTime(
    data=lat_long_weight,
    radius=10,
    index=index).add_to(time_heat_map)

time_heat_map

AttributeError: module 'datetime' has no attribute 'strptime'

In [182]:
rides.head()

,ride_id,line_id,from_id,to_id,from_stop_geoc,to_stop_geoc,capacity,num_buses,departure_date,departure_time,from_lat,from_long,to_lat,to_long
0,79143551,5328,24,11548,"{""latitude"":51.032368284265,""longitude"":6.9910...","{""latitude"":50.881001,""longitude"":7.117833}",78.0,1,2020-01-02,06:45:00,51.032368,6.991052,50.881001,7.117833
1,79143561,5328,25,52,"{""latitude"":51.2228649,""longitude"":6.7954195}","{""latitude"":51.450253,""longitude"":7.01461}",78.0,1,2020-01-01,20:40:00,51.222865,6.795420,51.450253,7.014610
2,79143561,5328,52,49,"{""latitude"":51.450253,""longitude"":7.01461}","{""latitude"":51.51954,""longitude"":7.459387}",78.0,1,2020-01-01,21:25:00,51.450253,7.014610,51.519540,7.459387
3,79143561,5328,251,1494,"{""latitude"":51.039905,""longitude"":13.730732}","{""latitude"":50.089617,""longitude"":14.440946}",78.0,1,2020-01-02,05:20:00,51.039905,13.730732,50.089617,14.440946
4,79143561,5328,30,251,"{""latitude"":51.345057312421,""longitude"":12.384...","{""latitude"":51.039905,""longitude"":13.730732}",78.0,1,2020-01-02,03:35:00,51.345057,12.384724,51.039905,13.730732


### Cap per bus as color

In [152]:
# initialize map
folium_map = folium.Map(location=[50.109900, 8.648916],
                        zoom_start=5,
                        tiles="CartoDB dark_matter")

# EXCLUDE TRAINS
# draw circle for all segments
for index, row in grouped_cap.query('cap_per_bus < 100').iterrows():
    
    radius = row['cap_per_bus']/10
    color = 'lime'
    
    marker = folium.CircleMarker(location=(
        row["from_lat"],
        row["from_long"]),
        radius=radius,
        color=color,
#         popup=row['cap_per_bus'], does not plot
        opacity=0.1,
        fill_opacity=0.2,
        fill=True
        )
    
    marker.add_to(folium_map)
folium_map

## Connections

In [ ]:
# https://blog.prototypr.io/interactive-maps-in-python-part-2-4943c4cb0ee4

## Animations

In [ ]:
# This first?
# https://blog.prototypr.io/interactive-maps-in-python-part-3-29f14a9b2f7d

In [154]:
def group_cap_per_hour(rides):
    rides['dep_hour'] = [x.hour for x in rides.departure_time]
    rides.groupby(
        ['from_id','from_lat','from_long','dep_hour'],
        as_index=False
    ).sum()
    
    return rides

In [155]:
hour_grouped_cap = group_cap_per_hour(rides)

In [36]:
# we could interpolate data to make the video more smooth, but first let it

In [156]:
def interpolate(df1, df2, x):
    """return a weighted average of two dataframes"""
    df = df1 * (1 - x) + df2 * x
    return df.replace(np.nan, 0)
  

def get_cap_by_minute(float_hour, hour_grouped_cap):
    """get an interpolated dataframe for any time, based
    on hourly data"""
    
    df1 = hour_grouped_cap.query(f"dep_hour=={int(float_hour)}")
    df2 = hour_grouped_cap.query(f"dep_hour=={int(float_hour)+1}")
    
    # add other columns
    df = interpolate(df1.loc[:,'capacity'], 
                     df2.loc[:,'capacity'], 
                     float_hour % 1)
    
    df["Start Station Name"] = df1["Start Station Name"]
    return df

In [136]:
plot_circles(df=hour_grouped_cap.query(f"dep_hour==8"),
             zoom_start=5,
             radius_scale=100)

In [151]:
def create_cap_frame(i, dep_hour, zoom_start=5, radius_scale=100):
    
    plot = plot_circles(
        df=hour_grouped_cap.query(f"dep_hour=={dep_hour}"),
        zoom_start=zoom_start,
        radius_scale=radius_scale)
    
    # generate the png file as a byte array
    png = plot._to_png()
    # create a PIL image object
    image = Image.open(io.BytesIO(png))
    draw = ImageDraw.ImageDraw(image)
    
    # now add a caption to the image to indicate the time-of-day.
    hour = int(dep_hour)
    minutes = int(0)
    
    # load a font
    font = ImageFont.truetype("RobotoCondensed-Light.ttf", 30)
    
    # draw time of day text
    draw.text((20,image.height - 50), 
              "time: {:0>2}:{:0>2}h".format(hour, minutes),
              fill=(255, 255, 255), 
              font=font)
    
    # draw title
    draw.text((image.width - 400,20), 
              "Offered Capacity per hour",
              fill=(255, 255, 255), 
              font=font)
    
    
    filename = os.path.join("animation/frame_{:0>5}.png".format(i))
    image.save(filename, "PNG")

In [152]:
for i, dep_hour in enumerate(np.arange(0,2,1)):
    create_cap_frame(i, dep_hour, zoom_start=5, radius_scale=100)